---
# Unit06: The Network and Array Methods

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter/IPython notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***
#### The code $\Downarrow$ BELOW $\Downarrow$ runs a notebook with other dependencies

In [ ]:
#------ Import Libraries
import os, sys
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)

#------ Magic commands
%matplotlib inline
%matplotlib widget

## Methods
One conceptual distinction between *array* and *network* is based on the spacing among array sensors and the wavelength of the signals.

- In the *array method* it is assumed some waveform coherence across different spatially-separated sensors.

- The *network method* assumes the waveforms at distinct sensors are uncorrelated.

- **The Network and Array Methods**
- Conceptually seismic data can be processed as a ’single station’, ’array’, and ’network’, in terms of how the spatially-separated sensors are used together, further conditioned by their signal and noise content. The crucial factor is the inter distance between sensors in terms of the waveform wavelength.
- In the ’array’ method one assumes some waveform coherence across different  sensors.
- Typically in the ’network’ method do not make this assumption, and treat waveforms at spatially-separated
sensors as uncorrelated- Conceptually seismic data can be processed as a ’single station’, ’array’, and ’network’, in terms of how the spatially-separated sensors are used together, further conditioned by their signal and noise content. The crucial factor is the inter distance between sensors in terms of the waveform wavelength.
- In the ’array’ method one assumes some waveform coherence across different  sensors.
- Typically in the ’network’ method do not make this assumption, and treat waveforms at spatially-separated
sensors as uncorrelated

<img src="./NetArr.png" width="300">

---
## Read a data set from TTB22
As ondas sísmicas geradas pela transferência da energia do fluxo turbulento no estuário do Rio Amazonas, resultante das suas grandes marés, por forças friccionais às rugosidades do seu fundo. O arranjo sísmico foi instalado na Ilha de Tatuoca, como tal o arranjo está circundado por uma distribuição de fontes no leito do rio. Fontes e geofones estão na superfície, tornando as ondas de superfície francamente dominantes em relação às de volume.
### The Experiment
- **The Receiver Array**
The array has 24 GS-20DX vertical geophones hooked to a $L=69$m cable, using takeouts spaced $\delta l=6$m from each other. The GS-20DX geophones have a natural frequency of $f_n=10\textrm{Hz}$, and a spurious frequency $f_{sp}>250$Hz. The array has a irregular circular shape, deployed in the Southern tip of the island, with its center at $\left(1^{\circ}12^{\prime}6.93^{\prime\prime}\textrm{S},48^{\circ}30^{\prime}23.39^{\prime\prime}\textrm{S}\right)$.

<img src="./ttb22.png" width="600">

- **The Geophones**
The GS-20DX vertical geophone has the following characteristics,
| :-: | :-: |
| Cut-off frequency | Spurious Frequency |
|$f_c=10$Hz | $f_{sp}>250$Hz|

(i) $f_{sp}$ is the resonance of the system perpendicular to this normal working axis; a combination of multiple modes of movement. $f_{Sp}$ should be above the anti--alias frequency of the recording system. The spurious frequency should not present a serious problem, unless in the presence of strong motion.

(ii) The natural and the spurious frequencies, $f_c$ and $f_{Sp}$, effectively set the limits of usable recording bandwidth of a geophone, due to the change of phase and a strong variation in sensitivity in the geophone output.

<img src="./GS-20DXa.png" width="600">

- **The data**
Each of the 12 traces of **file 3804** is $\Delta T=60$s long, with a sampling frequency of $f_{s}=250$Hz. This file was recorded on 2022-04-02, begining at 13h 56min 41s. That was during the maximum gradient of the local ebb tide.

<img src="./tide.png" width="500">


In [ ]:
"""
====================== A local routine to read a CSV file ======================
Read a cvs file and stores the information in an object numpy array
"""
def RGloc(filename):
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            data = []
            for line in lines[1:]:               # Skip header row
                parts = line.strip().split(',')
                data.append([int(parts[0]), float(parts[1]), float(parts[2])])
        return np.array(data, dtype=object)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None                                 
#
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
"""
#
#------ Read the seismic data
filename = '3804'
print(f">> Read with data file {filename}")
filename = '../Data/'+filename+'.dat'   
#------- Read the data file as a SEG2 object
st = read(filename)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f"1) Gather acquired on {st[0].stats.starttime}, has {len(st)} geophones along {dummy}m.")
dummy = (UTCDateTime(st[0].stats.endtime) - UTCDateTime(st[0].stats.starttime))
print(f"2) Each {dummy}s-long trace has {int(st[0].stats.npts)} data points.")
print(f"3) The sampling frequency is {st[0].stats.sampling_rate}Hz")
#
"""
====================== READ THE PHONES LOCATIONS ======================
"""
#------ Read the phones metric locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = RGloc('../Data/'+'ttb_loc.dat')
#
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

---
## Have a look on one trace.

In [ ]:
#------ Plot a randon phone
#dummy = np.random.randint(1, len(st)+1)
dummy = 1
print(f">> Plot phone {dummy}")
#st[dummy].plot(color='blue', type='relative', handle=True)
#
#------- Deep copy of the trace
trZ = st[dummy-1].copy()
tr0 = st[dummy-1].copy()
#
#--- relative time: nummpy array
time = trZ.times(type="relative")
#
#------ Plot Spectrogram 
p.Pspect(time, st[dummy-1])

---
## Filter out 60Hz spectral line
TTB 60Hz spectral line is around $l=60.015$Hz with an 1/2 width rejection band of $\delta l=0.8$Hz. This points to a rejection band of
$$\varDelta L=\left[59.2,60.8\right]$$
$$\underset{l-\delta l}{\cdots|\cdots}\cdots\cdots\underset{l}{|}\cdots\cdots\underset{l+\delta l}{\cdots|\cdots}$$


In [ ]:
#------ Notch 60Hz spectral line. Hint: bs 59.2 60.8
ent = input(f' Enter dflt [bs 59.2, 60.8], or enter your choice:  ')
ent = None if ent else ['bs', 59.2, 60.8]
#
trZ, ftype, flims = u.TrFlt(trZ, ent=ent)
#           +---+─> [fmin, fmax] = useful frequency range due to filter
print(f">> Useful frequency range after filtering [{flims[0]}, {flims[1]}]Hz")
#
#------ Plot Spectrogram
p.Pspect(time, trZ)
#
#-------- Save trace for the next cell + append to the processing flux
tr0, trZ, _ = u.AuxReset(tr0, trZ)

---
## Further Processing
### Filter the data
- Specify a bandpass filter considering the response of the sensor as well as the dataset constraints.
### Taper the data window
- Use the **Hanning Window**, which uses a cosine function to taper at both ends. Hint: 0.2.
### Have another look at the data...

In [ ]:
#
#-------- Filter the data. Hint: bp 5 50
ent = input(f' Enter dflt [bp 5. 50.], or enter your choice: ')
ent = None if ent else ['bp', 5., 50.]
#
trZ, ftype, flims = u.TrFlt(trZ, ent=ent)
#
#-------- Taper the data
ent = input(f' Enter the percentage to taper window ends (rtn=0.1)\n') or '0.1'
ent = float(ent)
# Apply Hanning on the signal
trZ.taper(type = 'hann', max_percentage = ent)
#
#------ Plot Spectrogram
p.Pspect(time, trZ)
#
#-------- Save trace for the next cell + append to the processing flux
tr0, trZ, _ = u.AuxReset(tr0, trZ)

---
## Display all the data as a seismogram
- Use the experience with one trace to process the whole stream together.
- A distance dependent plot shows the different move-out of seismic arrivals and gives an idea of the  backazimuth and slowness that could be expected.

In [ ]:
import pprint

print(f"Data is a {type(st)}, len={len(st)}, w/ att. trace({hasattr(st, 'traces')}) or data ({hasattr(st, 'data')})")

"""
====================== WORK WITH THE WHOLE GATHER ======================
"""
#Print the data mean. Use np.float64 to assure accuracy
print(f"\n The mean of 1st trace is {np.mean(st[0].data, dtype=np.float64)}\n")
#            trace data as a nummpy array   <────+───────+
# Remove any linear trend
st.detrend("linear")
# Remove the mean
st.detrend("demean")
#
#------ Notch 60Hz spectral line.
print(f">> Notch the data")
ent = input(f' Enter dflt [bs 59.2, 60.8], or enter your choice:  ')
ent = None if ent else ['bs', 59.2, 60.8]
st, ftype, flims = u.TrFlt(st, ent=ent)
#
#-------- Bandpass filter the data.
print("\r", end="")
print(f">> Bandpass filter the data")
ent = input(f' Enter dflt [bp 5. 50.], or enter your choice: ')
ent = None if ent else ['bp', 5., 50.]
#
st, ftype, flims = u.TrFlt(st, ent=ent)
print(f">> Useful range due to {ftype} filter: {flims[0]} to {flims[1]}Hz.")
#
#-------- Taper the data
print("\r", end="")
ent = input(f' Enter the percentage to taper window ends (rtn=0.1)\n') or '0.1'
ent = float(ent)
# Apply Hanning on the signal
print(f">> Window the data")
st.taper(type = 'hann', max_percentage = ent)
#
""" 
    Create a new stream and loop over traces. Add the distance information
    in the “header” and then add that trace to the stream.
"""
#------ Initialization
gather = Stream()
lon = 0.
lat = 0.
i   = 0
#
#------ Loop through traces
for t in st:
    tr = Trace(data=t.data)
    tr.stats.sampling_rate = t.stats.sampling_rate
    tr.stats.station       = f"gloc[i,0]"                           # Assign station name   
    tr.stats.starttime     = t.stats.starttime
    tr.stats.network       = "TTB22"                                # Assign network code
    tr.stats.channel       = "HHZ"                                  # Assign channel code
    tr.stats.location      = "0"                                    # Assign location code
    tr.stats.distance      = t.stats.seg2.RECEIVER_LOCATION         # Distance along cable
    tr.stats.coordinates = \
                             AttribDict({'latitude': ttb_gloc[i,1],
                                         'longitude': ttb_gloc[i,2],
                                         'elevation': 10.})
#
    lon += ttb_gloc[i,2]
    lat += ttb_gloc[i,1]
    i += 1
    gather += tr                                                      # gather.append(tr)
#
#------ Gather baricenter
print(f">> Gather baricenter form the field is: lat= -1.201925, lon= -48.506498; Datum  WGS84.")
lon /= float(i)
lat /= float(i)
print(f">> Gather baricenter here is: lat= {lat}, lon= {lon}.")
#
#------ Plot
gather.plot(type='section',
            scale=1.3, alpha=.7,
            orientation='horizontal')

In [ ]:
#
#------ Zoom in the seismogram
ent = input(f' Enter t0 and t1 to zoom: ')
ent = ent.rstrip().split(' ')
f0 = float(ent[0])
f1 = float(ent[1])
#
dt = gather[0].stats.starttime
gather.plot(type='section',
            scale=1.3, alpha=.7,
            starttime=dt+f0, endtime=dt+f1,
            orientation='horizontal')

## The Array Method
A wavefront arrives at the surface at an angle $i$ with the vertical. The wave propagates toward the surface with a velocity $v_{c}=\frac{\Delta s}{\varDelta t}$, with a horizontal component $v_{h}=\frac{\Delta x}{\varDelta t}$. By geometry, the inclination angle and distances are related through:

The direction-of-arrival (DOA) of a planar wavefront onto a seismic 2-D array can be described by the horizontal slowness, $u_h$, which is the inverse value of horizontal apparent velocity, $1/v_{h}$
$$
u_{h}=\frac{1}{v_{h}}=\frac{\sin i}{\left|\mathbf{v}_{c}\right|},
$$
being related to: (a) the angle of incidence $i$, (b) the true velocity $v_c$ and (c) its azimuth with the North *toward* the epicenter; the **baz** ($\theta$).
$$
\boldsymbol{U}_0 = (\frac{\sin\theta}{v_{h}},\frac{\cos\theta}{v_{h}},\frac{1}{v_{h}\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i)
                 = u_{h}(\sin\theta,\cos\theta,\frac{1}{\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i).
$$

<img src="./array1.png" width="600">

### Beamforming
The seismic signals at each sensor can be time-shifted and summed to enhance the S/N ratio by a factor of $\sqrt{N}$, the signal interfering constructively. If a given sensor location in relation to the geographic center of the array is $\boldsymbol{r}_{i},\,i=1,\ldots,N$, its relative time shift is
$$
\tau_{i}=\boldsymbol{r}_{i}.\boldsymbol{u}.
$$
The beamforming for the array is,
$$
		b\left(t\right)=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+
			\mathbf{r}_{i}\mathbf{\cdot u}\right)}=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+\tau_{i}\right)}.
$$

Beamforming can be used to enhance a given phase with a known backazimuth and slowness. Wrong backazimuths produce misleading waveforms.
### The TTB array

<img src="./ttb22.png" width="600">

In [ ]:
"""
====================== BEAMFORMING ======================
"""
# ---------- Use FK Analysis
out, stime, etime = u.BeamFK(st, flims, ttb_gloc)
#-- Make output human readable
t, rel_power, abs_power, baz, slow = out.T
#------------- print
sys.stdout.write('\n')
print(f'>> t       rel_power abs_power   baz(deg) slow(s/km)')    
for i in range(len(t)):
    print(f'   {round(t[i],4)}, {round(rel_power[i],4)}, {round(abs_power[i],4)}, {round(baz[i],4)}, {round(slow[i],4)}')    
#------------ Plot
p.pltbaz(out, stime , etime)

---
